In [1]:
# nbi:hide_in
import warnings
warnings.filterwarnings("ignore") # elimina il warning per pandas_datareader

import numpy as np 
import pandas as pd 

import pandas_datareader.data as web

import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from datetime import datetime
import time
import ipywidgets as wg
from ipywidgets.embed import embed_minimal_html

%matplotlib inline

In [2]:
# nbi:hide_in
start = '2010-04'
end = '2020-03'

ff_3f  = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0]

ff_48  = web.DataReader('48_Industry_Portfolios','famafrench', start, end)[0]

data = pd.concat([ff_3f, ff_48], axis=1, sort=False)

In [3]:
# nbi:hide_in
for i in range(4,len(data.columns)):
    data[data.columns[i]] = data[data.columns[i]] - data['RF']
new_data =data.drop(columns=['SMB','HML','RF'])
e_r = np.mean(new_data.iloc[:,1:])
x = sm.add_constant(new_data['Mkt-RF'])

In [5]:
# nbi:hide_in
window = 24 # rolling window

beta = {}
index = {}
window = 24 # rolling window
for i in range(len(new_data.index)-window-1):
    index[i] = 'Da ' + str(new_data.index[i]) + ' a ' + str(new_data.index[i+window]) # Per il titolo
    beta[i] = []
    for j in range(1, len(new_data.columns)):
        beta[i].append(sm.OLS(new_data.iloc[i:i+window,j], x.iloc[i:i+window]).fit().params[1])
        
def graph(t):
    '''I dati sono mensili, per cui 10 anni sono 120 mesi: la finestra rolling è 2 anni, quindi 24 mesi.
    Di conseguenza, ci saranno 120 - 24 = 96 regressioni rolling
    t indica quale tra queste regressioni viene scelta, dove t = 0 è la prima
    '''
    plt.figure(figsize=(15,8))
    ax = sns.regplot(beta[t],e_r)
    for i, txt in enumerate(e_r.index):
        ax.annotate(txt, (beta[t][i], e_r[i]))
    plt.ylabel('Expected Return')
    plt.xlabel('Beta')
    plt.title(index[t])

c_slide = wg.IntSlider(value=0, min=0, max=len(new_data.index)-window-1, step=1)
wg.interact(graph, t = c_slide)

interactive(children=(IntSlider(value=0, description='t', max=95), Output()), _dom_classes=('widget-interact',…